In [4]:
import xarray as xr
import fsspec
import pandas as pd
from seapopym.standard.units import StandardUnitsLabels
import hvplot.xarray

In [5]:
station = "Hot"

## Load DATA


### Observations


In [6]:
hot_obs = xr.open_dataset(f"../data/1_products/{station}_obs_zoo_climato_monthly_2002_2015.zarr", engine="zarr").load()
hot_obs

<xarray.Dataset> Size: 6kB
Dimensions:            (time: 155, latitude: 1, longitude: 1, layer: 1)
Coordinates:
  * latitude           (latitude) float64 8B 22.75
  * layer              (layer) int64 8B 0
  * longitude          (longitude) int64 8B -158
  * time               (time) datetime64[ns] 1kB 2002-01-31 ... 2014-11-30
Data variables:
    day                (time, latitude, longitude, layer) float64 1kB 1.397 ....
    day_lowess_0.15    (time, latitude, longitude, layer) float64 1kB 1.465 ....
    night              (time, latitude, longitude, layer) float64 1kB 2.237 ....
    night_lowess_0.15  (time, latitude, longitude, layer) float64 1kB 2.538 ....
Attributes:
    desc:     Aggregated and smoothed observations using lowess filtre.

In [7]:
time_start = hot_obs.time.data[0]
time_end = hot_obs.time.data[-1]

### CMEMS


In [8]:
hot_cmems = xr.open_dataset("../data/1_products/Hot_cmems.zarr", engine="zarr").load()
hot_cmems = hot_cmems.sel(time=slice(time_start, time_end))
hot_cmems

<xarray.Dataset> Size: 825kB
Dimensions:              (latitude: 1, longitude: 1, time: 4687, depth: 3)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 8B 22.75
  * longitude            (longitude) int64 8B -158
  * time                 (time) datetime64[ns] 37kB 2002-01-31 ... 2014-11-30
Data variables: (12/13)
    T                    (latitude, longitude, time, depth) float64 112kB 22....
    U                    (latitude, longitude, time, depth) float64 112kB 0.0...
    V                    (latitude, longitude, time, depth) float64 112kB -0....
    mnkc_epi             (latitude, longitude, time) float64 37kB 0.2991 ... ...
    mnkc_hmlmeso         (latitude, longitude, time) float64 37kB 0.8566 ... ...
    mnkc_lmeso           (latitude, longitude, time) float64 37kB 2.841 ... 1...
    ...                   ...
    mnkc_mumeso          (latitude, longitude, time) float64 37kB 0.5432 ... ...
    mnkc_umeso           (latitude, longitude, time) float64 37kB 0.3664 ... ...
    npp                  (latitude, longitude, time) float64 37kB 406.5 ... 3...
    pelagic_layer_depth  (latitude, longitude, time, depth) float64 112kB 104...
    zeu                  (latitude, longitude, time) float64 37kB 69.96 ... 7...
    zooc                 (latitude, longitude, time) float64 37kB 0.2757 ... ...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-10-17
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Global ocean low and mid trophic levels biomass content hin...

## Climato


In [9]:
hot_cmems_climato = hot_cmems.groupby("time.dayofyear").mean()

## Expand to original time series


In [10]:
climato_merged = {}

for var in hot_cmems.data_vars:
    if "depth" in list(hot_cmems[var].coords):
        data = []
        for depth in hot_cmems.depth.data:
            tab_1 = hot_cmems.time.dt.dayofyear.to_series()
            tab_2 = hot_cmems_climato[var].sel(depth=depth).to_series()
            data.append(pd.merge(tab_1, tab_2, on="dayofyear", how="left")[var])
        climato_merged[var] = xr.DataArray(
            data, dims=("depth", "time"), coords={"depth": hot_cmems.depth.data, "time": hot_cmems.time.data}
        )
    else:
        tab_1 = hot_cmems.time.dt.dayofyear.to_series()
        tab_2 = hot_cmems_climato[var].to_series()
        climato_merged[var] = xr.DataArray(
            pd.merge(tab_1, tab_2, on="dayofyear", how="left")[var],
            dims=("time"),
            coords={"time": hot_cmems.time.data},
        )

climato_merged = xr.Dataset(climato_merged)

climato_merged = climato_merged.expand_dims({"latitude": hot_cmems.latitude, "longitude": hot_cmems.longitude})
for var in climato_merged:
    climato_merged[var].attrs = hot_cmems[var].attrs
for coord in climato_merged.coords:
    climato_merged[coord].attrs = hot_cmems[coord].attrs
climato_merged

<xarray.Dataset> Size: 825kB
Dimensions:              (latitude: 1, longitude: 1, depth: 3, time: 4687)
Coordinates:
  * latitude             (latitude) float64 8B 22.75
  * longitude            (longitude) int64 8B -158
  * depth                (depth) int64 24B 0 1 2
  * time                 (time) datetime64[ns] 37kB 2002-01-31 ... 2014-11-30
Data variables: (12/13)
    T                    (latitude, longitude, depth, time) float64 112kB 23....
    U                    (latitude, longitude, depth, time) float64 112kB -0....
    V                    (latitude, longitude, depth, time) float64 112kB -0....
    mnkc_epi             (latitude, longitude, time) float64 37kB 0.3072 ... ...
    mnkc_hmlmeso         (latitude, longitude, time) float64 37kB 0.8849 ... ...
    mnkc_lmeso           (latitude, longitude, time) float64 37kB 2.279 ... 2...
    ...                   ...
    mnkc_mumeso          (latitude, longitude, time) float64 37kB 0.6114 ... ...
    mnkc_umeso           (latitude, longitude, time) float64 37kB 0.5152 ... ...
    npp                  (latitude, longitude, time) float64 37kB 331.4 ... 2...
    pelagic_layer_depth  (latitude, longitude, depth, time) float64 112kB 115...
    zeu                  (latitude, longitude, time) float64 37kB 76.89 ... 7...
    zooc                 (latitude, longitude, time) float64 37kB 0.2935 ... ...

## Plot


In [11]:
climato_merged.mean(["latitude", "longitude"]).hvplot.line(
    x="time",
    groupby="depth",
    title=f"Climato {station} - CMEMS T",
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'30f82eb4-7cda-43a9-a8b1-7e9f581cc5bd': {'version…

## Export


In [12]:
climato_merged.to_zarr(
    f"../data/1_products/{station}_cmems_climato_{pd.Timestamp(time_start).year}_{pd.Timestamp(time_end).year}.zarr",
    mode="w",
)